In [2]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.8 MB/s eta 0:00:00


In [17]:
import re
import numpy as np
import html
import ast
import contractions
import unicodedata
import pickle

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
with open('/content/drive/MyDrive/emojien/Data/emoji_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [8]:
tokenizer = loaded_data['tokeizer']
emoji_dict = loaded_data['emoji_dict']

model = load_model('/content/drive/MyDrive/emojien/model/model.h5')

In [32]:
def normalize_stretched_word(word):
    return re.sub(r'(.)\1+', r'\1\1', word)

lemmatizer = WordNetLemmatizer()

def lemmatize_and_normalize_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    normalized_tokens = [normalize_stretched_word(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in normalized_tokens]
    return ' '.join(lemmatized_tokens)

def preprocess(text):

    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = text.lower() # lowercase
    text = html.unescape(text) # remove html entities
    text =  re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove links
    text = re.sub(r'@', '', text) # remove @
    text = re.sub(r'/', ' ', text) # convert / to space
    text = contractions.fix(text) # convert contractions to single words
    text = re.sub(r'[^a-z\s]', '', text) # remove punc
    text = lemmatize_and_normalize_text(text) # lemmatize and normalize
    return text

def predict(text):
    text = preprocess(text)
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen = 47, padding='pre')
    pred = model.predict(token_list)

    top5 = np.argsort(pred[0])[-5:][::-1]

    pred_emojis = [ast.literal_eval(emoji_dict[i])[0] for i in top5]
    return pred_emojis

In [36]:
text = ""
pred = predict(text)
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
['🙏🏾', '🤤', '😋', '👏', '🚀']


In [43]:
test_sentences = [
    "I can't wait for this party tonight!",
    "Why does everything have to be so complicated?",
    "Just scored tickets to the concert of the year!",
    "I'm so pissed off at my boss right now.",
    "I need a drink after this long day.",
    "She looked absolutely stunning in that dress.",
    "I'm feeling pretty sick today, might have to stay in bed.",
    "Why do people have to be so fake?",
    "Just got back from the gym, feeling pumped!",
    "Can't stop thinking about what happened last night.",
    "My phone died in the middle of our conversation, so annoying!",
    "This food is so spicy, but I love it!",
    "Feeling like a total badass after that presentation.",
    "I'm so freaking bored right now.",
    "Had a wild night out with the crew, can't remember half of it.",
    "Why does my love life always have to be so messy?",
    "Just got my paycheck, time to treat myself!",
    "Ugh, I really need to hit the gym again.",
    "I can't believe he said that to me, what an asshole.",
    "Nothing better than a hot bath after a long day.",
    "WTF just happened? I’m so confused.",
    "Why do I always attract the crazies?",
    "Just had the best sex of my life.",
    "This traffic is driving me insane!",
    "Life is too short to deal with stupid people.",
    "I'm about to lose my mind, seriously.",
    "Why is everything so freaking expensive?",
    "Just finished binge-watching my favorite show.",
    "I'm so hungry I could eat a horse.",
    "Fuck, I can't believe I messed that up.",
    "Do you wanna fuck me"
]


In [44]:
for sentence in test_sentences:
    pred = predict(sentence)
    print(f"Sentence: {sentence}")
    print(f"Predicted Emojis: {pred}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Sentence: I can't wait for this party tonight!
Predicted Emojis: ['💗', '😌', '😩', '🖤', '😎']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Sentence: Why does everything have to be so complicated?
Predicted Emojis: ['😢', '😔', '💔', '😒', '🙄']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Sentence: Just scored tickets to the concert of the year!
Predicted Emojis: ['💫', '😱', '🙏', '🙌', '©']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Sentence: I'm so pissed off at my boss right now.
Predicted Emojis: ['🙃', '😒', '🙂', '😫', '😤']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Sentence: I need a drink after this long day.
Predicted Emojis: ['🙃', '😩', '😭', '😌', '😇']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sentence: She looked absolutely stunning in that dress.
Predicted Emojis: ['👌', '😀', '😍', '😱', '🤗']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sentence: I'm feeling pretty sick today, might have to stay in bed.
Predicted Emojis: ['😩', '🙃', '🤪', '😭', '😘']

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step